In [1]:
import numpy as np
import re
import glob

def process_txt(file_path, max_len=50):
    """
    处理单个txt文件，返回一个 2x50 的矩阵
    """
    x_vals, z_vals = [], []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            match = re.findall(r"x:([-+]?\d*\.?\d+),y:([-+]?\d*\.?\d+),z:([-+]?\d*\.?\d+)", line)
            if match:
                x, _, z = match[0]
                x_vals.append(float(x))
                z_vals.append(float(z))
    
    # 不够补 0
    x_vals = (x_vals + [0.0] * max_len)[:max_len]
    z_vals = (z_vals + [0.0] * max_len)[:max_len]

    return np.array([x_vals, z_vals])

# 批量读取文件
files_1=glob.glob(r"D:\新建文件夹 (5)\data\1_UP\*.txt")
files_2=glob.glob(r"D:\新建文件夹 (5)\data\2_DOWN\*.txt")
files_3=glob.glob(r"D:\新建文件夹 (5)\data\3_LEFT\*.txt")
files_4=glob.glob(r"D:\新建文件夹 (5)\data\4_RIGHT\*.txt")
matrices_1=[process_txt(f) for f in files_1]
matrices_2=[process_txt(f) for f in files_2]
matrices_3=[process_txt(f) for f in files_3]
matrices_4=[process_txt(f) for f in files_4]
#堆叠成三维数组 N×2×50
data_1= np.stack(matrices_1)
data_2= np.stack(matrices_2)
data_3= np.stack(matrices_3)
data_4= np.stack(matrices_4)

In [2]:
print(data_2.shape) 
print(data_2[0])

(100, 2, 50)
[[ 0.    0.    0.    0.   -0.   -0.01 -0.01 -0.04 -0.06 -0.05  0.01  0.09
   0.17  0.27  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.  ]
 [ 0.    0.    0.    0.    0.01  0.01  0.   -0.07 -0.16 -0.21 -0.24 -0.28
  -0.31 -0.37  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.  ]]


In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# 假设你的四个矩阵分别是 data1, data2, data3, data4，形状 (100, 2, 50)
# 先拼接成一个总数据集
X = np.concatenate([data_1, data_2, data_3, data_4], axis=0)  # (400, 2, 50)
y = np.array([1]*100 + [2]*100 + [3]*100 + [4]*100)      # (400,)

# 展平每个样本 (2,50) -> (100,)
X = X.reshape(X.shape[0], -1)  # (400, 100)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=44, stratify=y
)

clf = SVC(kernel='rbf', C=1, gamma=1.0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("准确率:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


准确率: 0.9416666666666667
              precision    recall  f1-score   support

           1       0.96      0.90      0.93        30
           2       0.97      1.00      0.98        30
           3       0.96      0.90      0.93        30
           4       0.88      0.97      0.92        30

    accuracy                           0.94       120
   macro avg       0.94      0.94      0.94       120
weighted avg       0.94      0.94      0.94       120



In [8]:
B_1=data_1.reshape(data_1.shape[0],-1)
B_2=data_2.reshape(data_2.shape[0],-1)
B_3=data_3.reshape(data_3.shape[0],-1)
B_4=data_4.reshape(data_4.shape[0],-1)

a_1=clf.predict(B_1)
a_2=clf.predict(B_2)
a_3=clf.predict(B_3)
a_4=clf.predict(B_4)

print(a_1)
print(a_2)
print(a_3)
print(a_4)

[1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 4 1 1 4 4 1 1 1 3 1 1 1 1 1 1 1]
[2 2 2 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 2 2 3 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[2 3 3 3 3 3 3 3 2 3 3 3 3 3 3 3 3 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]


In [10]:
import serial
import time
import os

#串口参数
SERIAL_PORT = "COM7"   # 根据实际修改，比如 Linux 下 "/dev/ttyUSB0"
BAUD_RATE = 115200

#保存路径
SAVE_DIR = r"D:\新建文件夹 (5)\data"
os.makedirs(SAVE_DIR, exist_ok=True)

ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1)
time.sleep(2)  # 等待串口稳定

recording = False
buffer = []

In [11]:
while True:
    line = ser.readline().decode("utf-8", errors="ignore").strip()
    if not line:
        continue
    if line == "===START===":
        print("开始采集样本")
        buffer = []
        recording = True
        continue
    if line == "===END===":
        if recording:
            filename = os.path.join(SAVE_DIR, "test.txt")
            with open(filename, "w") as f:
                f.write("\n".join(buffer))
            print("样本已保存")
            recording = False
        array=[process_txt(r"D:\新建文件夹 (5)\data\test.txt")]
        test=np.stack(array)
        test=test.reshape(test.shape[0],-1)
        result=clf.predict(test)
        print(result)
        if(result[0]==1):
            os.system("start cmd") 
        continue

    if recording:
        buffer.append(line)

开始采集样本
样本已保存
[3]
开始采集样本
样本已保存
[1]
开始采集样本
样本已保存
[3]
开始采集样本
样本已保存
[1]


KeyboardInterrupt: 

In [10]:
ser.close()

NameError: name 'ser' is not defined

In [9]:
from micromlgen import port
c_code = port(clf)
with open(r"D:\新建文件夹 (5)\data\classifier.h", "w") as f:
    f.write(c_code)

print("模型已导出到 classifier.h")

模型已导出到 classifier.h
